In [1]:
import re
import pprint
import csv
import pandas as pd
from openpyxl import load_workbook

### Obtener citas ID y titulo de un archivo bib

Con este script, podemos encontrar todas los identificadores utilizados para citas en un archivo .bib y auxiliarmente el título del paper.
Esto fue utilizado, junto con otro script, para mapear citas de LaTeX a citas hardcodeadas de un archivo word. 

In [2]:
def obtener_citas_con_titulo(archivo):
    """
    Cada cita dentro de un archivo.bib consiste en (ejemplo):
    
    @article{pepe_rapid_2020, 
        title={Rapid ...}, 
        ...
    }

    Esto pasará en cualquier archivo, siempre y cuando la cita se haya agregado algo prolija.
    En cualquier caso, recomiendo formatear el archivo para mayor robustez.
    """
    citas = set()
    with open(archivo, 'r') as f:
        for linea in f:
            linea = linea.strip()
            if linea.startswith("@"):
                type, id = linea.split("{")
                # removemos la coma final
                id = id[:-1]
            elif "title =" in linea:
                _, valor = linea.split("=", 1)
                # remover espacios adicionales
                valor = valor.strip()
                # remover coma final (para todos menos último campo)
                if valor[-1] == ',':
                    valor = valor[:-1]
                # remover llaves
                valor = valor[1:-1]
            elif linea == "}":
                citas.add((id, valor))
                id, valor = None, None
    return citas

In [4]:
archivo_exportar = 'citas_en_bib.xlsx'
archivo_de_citas = 'tesis.bib'

datos = []

citas = obtener_citas_con_titulo(archivo_de_citas)

for id_cita, titulo in citas:
    """
    Las citas exportadas por Zotero, generan una ID en (casi) siempre 'autor_titulo_año' donde 
    - autor: es el apellido del primer autor
    - titulo: es la primer palabra 'significativa' (ej 'The rapid ...' toma 'rapid' ignorando 'the')
    Hay excepciones en especial con autores que tienen mas de un paper el mismo año.
    """
    elementos = id_cita.split('_')
    autor = autor[0]
    año = año[-1]
    datos.append([id_cita, autor, año, titulo])

# Convertir la lista de datos a un DataFrame de pandas
df = pd.DataFrame(datos, columns=['id_cita', 'autor', 'año', 'titulo'])

df.sort_values(by='id_cita',inplace=True)

# Guardar el DataFrame en un archivo Excel
df.to_excel(archivo_exportar, index=False)